In [1]:
from dash import html, dcc, Dash, Input, Output
from jupyter_dash import JupyterDash
import plotly.express as px
import pandas as pd
import os

In [2]:
# Threshold for whether likely to conver or churn
THRESHOLD = 0.65

# If no files present, need to run pipeline for predictions (e.g. 'make all' or 'make sample')
pred_dir = "../../data/predictions/"
all_predictions = !ls -ls $pred_dir
prediction_file = all_predictions.pop().split()[-1]
print(prediction_file)
prediction_file_full_path = os.path.join(os.path.abspath(pred_dir), prediction_file)
prediction_file_full_path

08-17-22_0757_predictions.csv


'/home/bc/Development/capstone_project/data/predictions/08-17-22_0757_predictions.csv'

In [3]:
new_trialers = pd.read_csv(prediction_file_full_path, index_col=0)
new_trialers["color"] = new_trialers.ChurnProb.apply(
    lambda x: "green" if x < THRESHOLD else "red")

def build_out_md_string(row):
    if isinstance(row, pd.Series):
        frame = row.to_frame().T
    elif isinstance(row, pd.DataFrame):
        frame = row
    
    if len(frame) == 1:
        string_out = "--- | ---"
        frame = row
        for col in frame.columns:
            string_out += f"\n{col} | {row[col]}"
        return string_out
    elif len(frame) == 0:
        return "## Select user"        
    else:
        return '**Error, received more than one row of users**'

fig1 = px.scatter(
    new_trialers, 
    x='ChurnProb', 
    y="total_watchtime_day_1",
    facet_row='trial_length_days',
    labels={"trial_length_days": "Length of Trial",
            "total_watchtime_day_1": "Video watch time day 1", 
            "ChurnProb": "Probability of Churning"},
    color='color')

app = JupyterDash(__name__)
server = app.server

In [7]:
@app.callback(
    Output('table-1', 'children'),
    Input('graph-1', 'hoverData')
)
def update_graph_2(value):
    max_rows = 10
    print(value)
    if value:
        index = value['points'][0]['pointNumber']
        dataframe = new_trialers.iloc[index].reset_index().T
        print(dataframe)
        return html.Table(
            html.Thead(
                html.Tr([html.Th(col) for col in dataframe.columns])
            ),
            html.Tbody([
                html.Tr([
                    html.Td(dataframe.iloc[i][col]) for col in dataframe.columns
                ]) for i in range(min(len(dataframe), max_rows))
            ])
        )
    else:
        return None

In [8]:
app.layout = html.Div(
    [
        html.H1(
            'Skillshare Churn Predictions',
            style={
                'textAlign': 'center',
                'color': 'black',
                "background": "white"}),
        dcc.Graph(
            id='graph-1',
            figure=fig1,
            style={'width':'400px'}),
        html.Table(id='table-1'),
    ])
    # style={
    #     "background": "#000080"})

In [9]:
app.run_server(mode='inline', debug=True)

/usr/lib/python3.10/site-packages/jupyter_dash/jupyter_app.py:139: UserWarning:

The 'environ['werkzeug.server.shutdown']' function is deprecated and will be removed in Werkzeug 2.1.

